In [ ]:
%%capture

!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
#!pip install git+https://github.com/huggingface/peft.git
#!pip install git+https://github.com/huggingface/trl.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!nvidia-smi

In [ ]:
import torch
cuda_id = 0 # 0 for google colab and 1 for Titan XP on local computer
device = torch.device("cuda:%s" % cuda_id if torch.cuda.is_available() else "cpu")
device_name = torch.cuda.get_device_name(cuda_id) if torch.cuda.is_available() else "cpu"
print("We are using the device %s - %s" % (device, device_name))

We are using the device cuda:0 - Tesla V100-SXM2-16GB


In [ ]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/content/cache
%env HF_DATASETS_CACHE=/content/cache
%env CUDA_LAUNCH_BLOCKING=1

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/content/cache
env: HF_DATASETS_CACHE=/content/cache
env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys

In [ ]:
from datasets import load_dataset

swda = load_dataset("swda")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/213543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/56729 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4514 [00:00<?, ? examples/s]

In [ ]:
print(swda)

DatasetDict({
    train: Dataset({
        features: ['swda_filename', 'ptb_basename', 'conversation_no', 'transcript_index', 'act_tag', 'damsl_act_tag', 'caller', 'utterance_index', 'subutterance_index', 'text', 'pos', 'trees', 'ptb_treenumbers', 'talk_day', 'length', 'topic_description', 'prompt', 'from_caller', 'from_caller_sex', 'from_caller_education', 'from_caller_birth_year', 'from_caller_dialect_area', 'to_caller', 'to_caller_sex', 'to_caller_education', 'to_caller_birth_year', 'to_caller_dialect_area'],
        num_rows: 213543
    })
    validation: Dataset({
        features: ['swda_filename', 'ptb_basename', 'conversation_no', 'transcript_index', 'act_tag', 'damsl_act_tag', 'caller', 'utterance_index', 'subutterance_index', 'text', 'pos', 'trees', 'ptb_treenumbers', 'talk_day', 'length', 'topic_description', 'prompt', 'from_caller', 'from_caller_sex', 'from_caller_education', 'from_caller_birth_year', 'from_caller_dialect_area', 'to_caller', 'to_caller_sex', 'to_caller_educ

In [ ]:
print(swda["train"])

Dataset({
    features: ['swda_filename', 'ptb_basename', 'conversation_no', 'transcript_index', 'act_tag', 'damsl_act_tag', 'caller', 'utterance_index', 'subutterance_index', 'text', 'pos', 'trees', 'ptb_treenumbers', 'talk_day', 'length', 'topic_description', 'prompt', 'from_caller', 'from_caller_sex', 'from_caller_education', 'from_caller_birth_year', 'from_caller_dialect_area', 'to_caller', 'to_caller_sex', 'to_caller_education', 'to_caller_birth_year', 'to_caller_dialect_area'],
    num_rows: 213543
})


In [ ]:
print(swda["train"][0])

{'swda_filename': 'sw00utt/sw_0001_4325.utt', 'ptb_basename': '4/sw4325', 'conversation_no': 4325, 'transcript_index': 0, 'act_tag': 115, 'damsl_act_tag': 26, 'caller': 'A', 'utterance_index': 1, 'subutterance_index': 1, 'text': 'Okay.  /', 'pos': 'Okay/UH ./.', 'trees': '(INTJ (UH Okay) (. .) (-DFL- E_S))', 'ptb_treenumbers': '1', 'talk_day': '03/23/1992', 'length': 5, 'topic_description': 'CHILD CARE', 'prompt': 'FIND OUT WHAT CRITERIA THE OTHER CALLER WOULD USE IN SELECTING CHILD CARE SERVICES FOR A PRESCHOOLER.  IS IT EASY OR DIFFICULT TO FIND SUCH CARE?', 'from_caller': 1632, 'from_caller_sex': 'FEMALE', 'from_caller_education': 2, 'from_caller_birth_year': 1962, 'from_caller_dialect_area': 'WESTERN', 'to_caller': 1519, 'to_caller_sex': 'FEMALE', 'to_caller_education': 1, 'to_caller_birth_year': 1971, 'to_caller_dialect_area': 'SOUTH MIDLAND'}


In [ ]:
COLUMNS_TO_KEEP = ["swda_filename", "ptb_basename", "conversation_no", "transcript_index", "act_tag", "text", "damsl_act_tag", "caller", "utterance_index", "subutterance_index"]
all_columns = swda["train"].column_names
columns_to_remove = set(all_columns) - set(COLUMNS_TO_KEEP)

swda = swda.remove_columns(columns_to_remove)